In [10]:
import time
import requests
import pandas as pd
from io import StringIO
from pathlib import Path

BASE_URL = "https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min"

AUTH_KEY = "bWb9TUUATEGm_U1FAExBlg"
STN = "368"

OUT_CSV = Path("AWS_368.csv")
N_CALLS = 1000 # 총 1000번

WINDOW_HOURS = 20 # 12시간

STEP_MINUTES = 1 # 마지막 관측시각 + 1분

TIMEOUT_SEC = 60 # 60초 동안 기다림
SLEEP_SEC = 0.5 # 너무 빠르면 막힐 수 있어 약간 쉬기(필요 시 조절)

COLS = [
    "YYMMDDHHMI", "STN",
    "WD1", "WS1", "WDS", "WSS",
    "WD10", "WS10",
    "TA", "RE",
    "RN_15m", "RN_60m", "RN_12H", "RN_DAY",
    "HM", "PA", "PS", "TD"
]

In [11]:
def fetch_text(tm1: str, tm2: str) -> str:
    params = {
        "tm1": tm1,
        "tm2": tm2,
        "stn": STN,
        "disp": "1",
        "help": "0",
        "authKey": AUTH_KEY
    }
    r = requests.get(BASE_URL, params=params, timeout=TIMEOUT_SEC)
    r.raise_for_status()
    return r.text

In [12]:
def parse_kma_text_to_df(text: str) -> pd.DataFrame:
    # 1) 주석/빈줄 제거
    lines = []
    for ln in text.splitlines():
        ln = ln.strip()
        if not ln or ln.startswith("#"):
            continue
        lines.append(ln)

    if not lines:
        return pd.DataFrame(columns=COLS)

    # 2) 끝 토큰 '=' 제거
    cleaned = []
    for ln in lines:
        if ln.endswith(",="):
            ln = ln[:-2]
        elif ln.endswith("="):
            ln = ln[:-1]
            if ln.endswith(","):
                ln = ln[:-1]
        cleaned.append(ln)

    df = pd.read_csv(StringIO("\n".join(cleaned)), header=None, dtype=str)

    # 열수 체크
    if df.shape[1] != len(COLS):
        # 포맷이 바뀌었거나 에러 메시지가 섞인 경우가 흔함
        raise ValueError(f"컬럼 수 불일치: expected={len(COLS)}, got={df.shape[1]}")

    df.columns = COLS

    # datetime 파생
    df["datetime"] = pd.to_datetime(df["YYMMDDHHMI"], format="%Y%m%d%H%M", errors="coerce")

    # 숫자 변환
    for c in df.columns:
        if c in ("YYMMDDHHMI", "datetime"):
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # 결측 센티넬(-99.x) 처리
    num_cols = [c for c in df.columns if c not in ("YYMMDDHHMI", "datetime")]
    for c in num_cols:
        df.loc[df[c] <= -99, c] = pd.NA

    return df

In [13]:
def dt_to_tm(dt: pd.Timestamp) -> str:
    # KST 타임존을 별도로 붙이지 않아도, 문자열 기준으로 굴리면 충분함(네 데이터가 KST임)
    return dt.strftime("%Y%m%d%H%M")

In [14]:
def append_df_to_csv(df: pd.DataFrame, path: Path):
    if df.empty:
        return
    write_header = not path.exists()
    df.to_csv(path, mode="a", index=False, encoding="utf-8-sig", header=write_header)


In [15]:
def infer_next_tm1_from_existing_csv(path: Path, fallback_tm1: str) -> str:
    if not path.exists():
        return fallback_tm1

    # 큰 파일이면 tail 방식이 좋지만, 여기선 간단하게 마지막 행만 읽음
    last = pd.read_csv(path, usecols=["YYMMDDHHMI"]).tail(1)
    if last.empty:
        return fallback_tm1

    last_dt = pd.to_datetime(last["YYMMDDHHMI"].iloc[0], format="%Y%m%d%H%M", errors="coerce")
    if pd.isna(last_dt):
        return fallback_tm1

    next_dt = last_dt + pd.Timedelta(minutes=STEP_MINUTES)
    return dt_to_tm(next_dt)

In [ ]:
INITIAL_TM1 = "202408010000"  # 첫 실행용 기본값
tm1 = infer_next_tm1_from_existing_csv(OUT_CSV, INITIAL_TM1)

for i in range(N_CALLS):
    start_dt = pd.to_datetime(tm1, format="%Y%m%d%H%M")
    end_dt = start_dt + pd.Timedelta(hours=WINDOW_HOURS)
    tm2 = dt_to_tm(end_dt)

    print(f"[{i+1}/{N_CALLS}] tm1={tm1} tm2={tm2}")

    try:
        text = fetch_text(tm1, tm2)
        df = parse_kma_text_to_df(text)

        # 혹시 중복이 생기면(이어받기/겹침) 제거
        if not df.empty:
            df = df.drop_duplicates(subset=["YYMMDDHHMI", "STN"], keep="last")

        append_df_to_csv(df, OUT_CSV)

        # 다음 tm1 = 이번 응답의 마지막 datetime + 1분
        if df.empty or df["datetime"].isna().all():
            # 데이터가 비었으면 그냥 윈도우를 앞으로 한 칸 이동(12시간)
            tm1 = dt_to_tm(end_dt + pd.Timedelta(minutes=STEP_MINUTES))
        else:
            last_dt = df["datetime"].max()
            tm1 = dt_to_tm(last_dt + pd.Timedelta(minutes=STEP_MINUTES))

    except Exception as e:
        # 에러 나면 너무 멀리 점프하지 말고, 일단 12시간 뒤로만 이동해서 계속
        print("  [error]", repr(e))
        tm1 = dt_to_tm(end_dt + pd.Timedelta(minutes=STEP_MINUTES))

    time.sleep(SLEEP_SEC)

print("DONE:", OUT_CSV.resolve())

[1/1000] tm1=202408010000 tm2=202408012000
[2/1000] tm1=202408011604 tm2=202408021204
[3/1000] tm1=202408020808 tm2=202408030408
[4/1000] tm1=202408030141 tm2=202408032141
[5/1000] tm1=202408031803 tm2=202408041403
[6/1000] tm1=202408041020 tm2=202408050620
[7/1000] tm1=202408042220 tm2=202408051820
[8/1000] tm1=202408051821 tm2=202408061421
[9/1000] tm1=202408061224 tm2=202408070824
[10/1000] tm1=202408070349 tm2=202408072349
  [error] HTTPError('504 Server Error: Gateway Timeout for url: https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm1=202408070349&tm2=202408072349&stn=368&disp=1&help=0&authKey=bWb9TUUATEGm_U1FAExBlg')
[11/1000] tm1=202408072350 tm2=202408081950
[12/1000] tm1=202408080414 tm2=202408090014
[13/1000] tm1=202408081333 tm2=202408090933
[14/1000] tm1=202408090134 tm2=202408092134
[15/1000] tm1=202408091758 tm2=202408101358
[16/1000] tm1=202408100648 tm2=202408110248
[17/1000] tm1=202408102256 tm2=202408111856
[18/1000] tm1=202408111655 tm2=202408121255
[19/